In [ ]:
import numpy as np
base_seed = 1300
n_seeds = 10

model_id = 'lv'
save_path = '/home/david/code/lfi-experiments/snpec/results/' + model_id

use_maf = False

noise_sd = 50.0  # compare to log(noise_sd) = 2.3 in Owen et al. 2014, Likelihood free inference for Makov Processes: a comparison

include_initial_state = True

# simulation setup
setup_opts = {        
    'n_hiddens': [50, 50],
    'reg_lambda': 0.01,
    'pilot_samples': 1000,
    'verbose': True,
    'prior_norm': False,    
    'svi': False,
    'n_rnn': 100,
    'n_inputs_rnn': 2
}

run_opts = {
    'n_train': 1000,
    'n_rounds': 10,
    'minibatch': 100,
    'epochs': 2000,
    'moo': 'resample',
    'proposal': 'gaussian',
    'n_null': None,
    'train_on_all': True,
    'max_norm': 0.1,
    'val_frac': 0.1,
    'silent_fail': False,
    'reuse_prior_samples': True,
}
  
if run_opts['train_on_all']:
    run_opts['epochs'] = [run_opts['epochs'] // (r+1) for r in range(run_opts['n_rounds'])]

if use_maf:
    # control MAF seed
    rng_maf = np.random
    
    setup_opts.update({
        'mode': 'random',
        'n_mades': 5,
        'rng': rng_maf,
        'act_fun': 'tanh',
        'batch_norm': False
    })

pars_true = np.array([-4.60517019, -0.69314718,  0.        , -4.60517019])  # from SNL paper
#x0 was generated by simulation using the true parameters from the SNL paper, initial state not removed, dt=0.2
raw_data = np.array([50., 100., 59., 104.,  66., 117.,  81., 133.,  97., 131., 109., 135., 127.,
       132., 148., 135., 163., 125., 188., 108., 217.,  95., 239.,  75.,
       236.,  60., 235.,  46., 228.,  34., 217.,  28., 211.,  18., 193.,
        12., 175.,   8., 171.,   9., 162.,   5., 151.,   5., 136.,   6.,
       128.,   6., 111.,   8., 110.,   8.,  96.,   8.,  95.,   9.,  93.,
         9.,  81.,  10.,  80.,  11.,  78.,  17.,  74.,  17.,  72.,  19.,
        66.,  20.,  67.,  25.,  64.,  29.,  54.,  30.,  55.,  45.,  54.,
        45.,  53.,  48.,  48.,  50.,  54.,  65.,  61.,  75.,  64.,  77.,
        65.,  95.,  73., 106.,  85., 107.,  88., 114., 104., 113., 130.,
       114., 159., 110., 173.,  95., 196.,  77., 220.,  72., 219.,  49.,
       212.,  33., 209.,  24., 196.,  18., 188.,  18., 176.,  17., 166.,
        15., 158.,  14., 154.,  10., 138.,  13., 128.,  11., 119.,  11.,
       107.,   9.,  99.,   8.,  94.,  11.,  86.,  11.,  81.,  12.,  78.,
        15.,  67.,  19.,  63.,  16.,  58.,  20.,  58.,  28.,  57.,  36.,
        53.,  39.,  55.,  44.,  53.,  44.,  55.,  44.,  54.,  49.,  54.,
        66.,  54.,  77.,  53.,  93.,  64.,  98.,  76., 100.,  94., 110.,
       116., 112., 136., 102., 149., 105., 173., 103., 200.,  95., 208.,
        69., 224.,  54., 232.,  47., 217.,  40., 206.,  29., 202.,  24.,
       196.,  24., 181.,  23., 167.,  13., 158.,  14., 147.,   9., 135.,
         8., 121.,   9., 111.,   8.,  95.,  10.,  84.,  10.,  73.,  11.,
        68.,  14.,  61.,  16.,  57.,  20.,  50.,  19.,  46.,  25.,  43.,
        33.,  41.,  38.,  42.,  40.,  45.,  50.,  48.,  57.,  48.,  60.,
        47.,  63.,  49.,  68.,  48.,  66.,  42.,  71.,  43.,  84.,  46.,
        81.,  40.,  89.,  45., 121.,  55., 134.,  67., 136.,  83., 135.,
       100., 142., 129., 135., 148., 134., 163., 130., 182., 111., 195.,
       106., 206.,  93., 216.,  67., 224.,  47., 207.,  33., 202.,  24.,
       193.,  20., 182.,  14., 174.,   8., 161.,   8., 152.,   4., 140.,
         6., 131.,   5.])
raw_data += np.random.randn(*raw_data.shape) * noise_sd
obs = raw_data if include_initial_state else raw_data[2:]

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
import timeit
from copy import deepcopy
from delfi.utils.progress import no_tqdm, progressbar

from delfi.utils.viz import plot_pdf
import delfi.inference as infer
import delfi.distribution as dd
import delfi.generator
from delfi.summarystats import Identity

from lfimodels.snl_exps.util import save_results, load_results, StubbornGenerator, stubborn_defaultrej
from lfimodels.snl_exps.util import  save_results_byname, load_results_byname
from lfimodels.snl_exps.util import init_g_lv as init_g
from lfimodels.snl_exps.util import load_setup_lv as load_setup
from lfimodels.snl_exps.util import load_gt_lv as load_gt
from lfimodels.snl_exps.util import calc_all_lprob_errs
from lfimodels.snl_exps.LotkaVolterra import LotkaVolterra, LotkaVolterraStats
import snl.simulators.lotka_volterra as sim_lv
from snl.util.plot import plot_hist_marginals

print('pars_true : ', pars_true)

dt = 0.2

In [ ]:
def plot_gaussian_sequence(x, ps, **kwargs):
    m = np.array([p.xs[0].m[k] for p in ps])
    s = np.array([np.sqrt(p.xs[0].S[k, k]) for p in ps])
    return plt.fill_between(x, m - s, m + s, **kwargs)

# compare rnn and stats with and without no noise

In [ ]:
import os

exp_ids_rnn_nonoise = [d for d in os.listdir(save_path) if d.startswith('rnn_nonoise_seed23')]
prnn_nonoise = []
for exp_id in exp_ids_rnn_nonoise:
    try:
        pstats, Vstats = load_results_byname(exp_id=exp_id, path=save_path)
    except:
        continue
    del Vstats
    if type(pstats[0]) is list:        
        #assert len(pstats) == 1
        pstats = pstats[-1]
    prnn_nonoise.append(pstats)

In [ ]:
exp_ids_stats_nonoise = [d for d in os.listdir(save_path) if d.startswith('stats_fc_nonoise_seed23')]
pstats_nonoise = []
for exp_id in exp_ids_stats_nonoise:
    try:
        pstats, Vstats = load_results_byname(exp_id=exp_id, path=save_path)
    except:
        continue
    del Vstats
    if type(pstats[0]) is list:
        #assert len(pstats) == 1
        pstats = pstats[-1]
    pstats_nonoise.append(pstats)

In [ ]:
exp_ids_rnn_noise = [d for d in os.listdir(save_path) if d.startswith('rnn_noise_seed13')]
prnn_noise = []
for exp_id in exp_ids_rnn_noise:
    try:
        pstats, Vstats = load_results_byname(exp_id=exp_id, path=save_path)
    except:
        continue
    del Vstats
    if type(pstats[0]) is list:
        #assert len(pstats) == 1
        pstats = pstats[-1]
    prnn_noise.append(pstats)

In [ ]:
exp_ids_stats_noise = [d for d in os.listdir(save_path) if d.startswith('stats_fc_noise_seed13')]
pstats_noise = []
for exp_id in exp_ids_stats_noise:
    try:
        pstats, Vstats = load_results_byname(exp_id=exp_id, path=save_path)
    except:
        continue
    del Vstats
    if type(pstats[0]) is list:
        assert len(pstats) == 1
        pstats = pstats[0]
    pstats_noise.append(pstats)

In [ ]:
alpha = 0.1
rnn_color = 'teal'
stats_color = 'violet'

dim_param = pars_true.size
figh = plt.figure(figsize=(16,6))
for k in range(dim_param):
    ax = plt.subplot(2, dim_param, k+1)    
    for i, pps in enumerate(pstats_nonoise):
        nsamples = run_opts['n_train'] * (1.0 + np.arange(len(pps)))
        hstats = plot_gaussian_sequence(nsamples, pps, color=stats_color, alpha=alpha)#, label='stats')
        plt.plot(nsamples, [p.xs[0].mean[k] for p in pps], color=stats_color)
    for i, pps in enumerate(prnn_nonoise):
        nsamples = run_opts['n_train'] * (1.0 + np.arange(len(pps)))
        hrnn = plot_gaussian_sequence(nsamples, pps, color=rnn_color, alpha=alpha)#, label='rnn')
        plt.plot(nsamples, [p.xs[0].mean[k] for p in pps], color=rnn_color)
    plt.plot(plt.xlim(), np.ones(2) * pars_true[k],'k')
    plt.ylim(pars_true[k] + np.array([-1., 1.]) * .5)
    plt.xlim([0, 10000])
    plt.xlabel('Simulations')
    plt.ylabel('$\\theta_{0}$'.format(k))
    ax.set_aspect(10000.)
    if k == 0:
        plt.legend(loc='lower right')
        plt.title('Observationn noise $\\sigma = 0$')

    ax = plt.subplot(2, dim_param, k+1 + dim_param)    
    for i, pps in enumerate(pstats_noise):
        nsamples = run_opts['n_train'] * (1.0 + np.arange(len(pps)))
        hstats = plot_gaussian_sequence(nsamples, pps, color=stats_color, alpha=alpha)#, label='stats')
        plt.plot(nsamples, [p.xs[0].mean[k] for p in pps], color=stats_color)
    for i, pps in enumerate(prnn_noise):
        nsamples = run_opts['n_train'] * (1.0 + np.arange(len(pps)))
        hrnn = plot_gaussian_sequence(nsamples, pps, color=rnn_color, alpha=alpha)#, label='rnn')
        plt.plot(nsamples, [p.xs[0].mean[k] for p in pps], color=rnn_color)
    plt.plot(plt.xlim(), np.ones(2) * pars_true[k],'k')
    plt.ylim(pars_true[k] + np.array([-1., 1.]) * 5)
    plt.xlim([0, 10000])
    plt.xlabel('Simulations')
    plt.ylabel('$\\theta_{0}$'.format(k))
    ax.set_aspect(1000.)
    if k == 0:
        plt.legend(loc='lower right')
        plt.title('Observationn noise $\\sigma = 50$')
figh.tight_layout()

In [ ]:
for ii in range(10):

    print('\n')
    print(str(ii))
    print('\n')
    
    round_to_plot = 10
    g=g_rnn

    g2 = deepcopy(g)
    g2.proposal = prnn_nonoise[ii][round_to_plot - 1]
    samples_rnn= np.array(g2.draw_params(5000)) 

    fig = plot_hist_marginals(samples_rnn,
                            gt=pars_true.flatten(),
                            lims=[[-5,2],[-5,2],[-5,2],[-5,2]])
    fig.show()


    g2 = deepcopy(g)
    g2.proposal = pstats_nonoise[ii][round_to_plot - 1]
    samples_ss =  np.array(g2.draw_params(5000)) 

    fig = plot_hist_marginals(samples_ss,
                            gt=pars_true.flatten(),
                            lims=[[-5,2],[-5,2],[-5,2],[-5,2]])
    fig.show()



"""
g2 = deepcopy(g)
g2.proposal = prnn_noise[ii][round_to_plot - 1]
samples_noise = np.array(g2.draw_params(5000)) 

_ = plot_hist_marginals(samples_noise,
                        gt=pars_true.flatten(),
                        lims=[[-5,2],[-5,2],[-5,2],[-5,2]])
fig.show()
"""


In [ ]:
import scipy
figh = plt.figure(figsize=(16,6))

round_plot = 10
for k in range(dim_param):
    xx = pars_true[k] + np.linspace(-1., 1., 5000) * 2
    
    plt.subplot(2, 4, k + 1)
    for p in pstats_nonoise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=stats_color)
    plt.xlabel('$\\theta_{0}$'.format(k))
    if k == 0:
        #plt.ylabel('$q_{F(x,\\phi)}(\\theta)$')
        plt.ylabel('$\\sigma = 0$')
        
    for p in prnn_nonoise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=rnn_color)
        
    plt.subplot(2, 4, 4 + k + 1)
    for p in pstats_noise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=stats_color)
    plt.xlabel('$\\theta_{0}$'.format(k))
    if k == 0:
        #plt.ylabel('$q_{F(x,\\phi)}(\\theta)$')
        plt.ylabel('$\\sigma = 50$')

        
    for p in prnn_noise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=rnn_color)
        
for k in range(dim_param):
    plt.subplot(2, 4, k + 1)
    YL = plt.ylim()
    plt.plot(pars_true[k] * np.ones(2), YL, 'r')
    plt.ylim(YL)
    plt.subplot(2, 4, 4 + k + 1)
    plt.plot(pars_true[k] * np.ones(2), plt.ylim(), 'r')


In [ ]:
fill_alpha = 0.2

max_rounds = 10

plt.figure()


y = np.hstack([np.array([-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps])[:max_rounds] for ps in prnn_nonoise])
m_rnn_nonoise = y.mean(axis=1)
s_rnn_nonoise = y.std(axis=1)
y = np.hstack([np.array([-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps])[:max_rounds] for ps in prnn_noise])
m_rnn_noise = y.mean(axis=1)
s_rnn_noise = y.std(axis=1)
y = np.hstack([np.array([-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps])[:max_rounds] for ps in pstats_nonoise])
m_stats_nonoise = y.mean(axis=1)
s_stats_nonoise = y.std(axis=1)
y = np.hstack([np.array([-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps])[:max_rounds] for ps in pstats_noise])
m_stats_noise = y.mean(axis=1)
s_stats_noise = y.std(axis=1)

plt.subplot(1, 2, 1)

m, s, c = m_stats_nonoise, s_stats_nonoise, stats_color
plt.fill_between(nsamples, m-s, m + s, color=c, alpha=fill_alpha)

m, s, c = m_rnn_nonoise, s_rnn_nonoise, rnn_color
plt.fill_between(nsamples, m-s, m + s, color=c, alpha=fill_alpha)

plt.plot(nsamples, m_stats_nonoise, color=stats_color, marker='.')
plt.plot(nsamples, m_rnn_nonoise, color=rnn_color,marker='.')

plt.xlabel('samples')
plt.ylabel('-log p(true params)')
plt.title('noise 0')
plt.xlim([0, 10000])
plt.ylim([-5, 40])

plt.subplot(1, 2, 2)
m, s, c = m_stats_noise, s_stats_noise, stats_color
plt.fill_between(nsamples, m-s, m + s, color=c, alpha=fill_alpha)

m, s, c = m_rnn_noise, s_rnn_noise, rnn_color
plt.fill_between(nsamples, m-s, m + s, color=c, alpha=fill_alpha)

plt.plot(nsamples, m_stats_noise, color=stats_color,marker='.')
plt.plot(nsamples, m_rnn_noise, color=rnn_color, marker='.')

plt.xlabel('samples')
plt.ylabel('-log p(true params)')
plt.title('noise 50')
plt.xlim([0, 10000])
plt.ylim([-5, 40])

In [ ]:
plt.figure()
for i, ps in enumerate(prnn_nonoise):
    nsamples = run_opts['n_train'] * (1.0 + np.arange(len(ps)))
    plt.plot(nsamples, [-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps], color=rnn_color)
for i, ps in enumerate(pstats_nonoise):
    nsamples = run_opts['n_train'] * (1.0 + np.arange(len(ps)))
    plt.plot(nsamples, [-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps], color=stats_color)    
plt.title('noise 0')
plt.xlim([0, 10000])
plt.ylim([-5, 20])

plt.figure()
for i, ps in enumerate(prnn_noise):
    nsamples = run_opts['n_train'] * (1.0 + np.arange(len(ps)))
    plt.plot(nsamples, [-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps], color=rnn_color)
for i, ps in enumerate(pstats_noise):
    nsamples = run_opts['n_train'] * (1.0 + np.arange(len(ps)))
    plt.plot(nsamples, [-pp.eval(pars_true.reshape(1, -1), log=True) for pp in ps], color=stats_color)    
plt.title('noise 50')
plt.xlim([0, 10000])
plt.ylim([-5, 50])

In [ ]:
seed=42

n_steps_used=(raw_data.shape[0] - 2)// 2
assert (n_steps_used + int(include_initial_state)) * 2 == obs.shape[0]
model = LotkaVolterra(dt=dt, duration=dt * n_steps_used, seed=seed, max_n_steps=50000, noise_sd=noise_sd)
prior = dd.Uniform(lower= [-5,-5,-5,-5], upper = [2,2,2,2], seed=seed)
summary = Identity(seed=seed)
g_rnn = StubbornGenerator(model=model, prior=prior, summary=summary, seed=seed)

n_steps_used=(raw_data.shape[0] - 2)// 2
assert (n_steps_used + int(include_initial_state)) * 2 == obs.shape[0]
model = LotkaVolterra(dt=dt, duration=dt * n_steps_used, seed=seed, max_n_steps=50000, noise_sd=noise_sd)
prior = dd.Uniform(lower= [-5,-5,-5,-5], upper = [2,2,2,2], seed=seed)
summary = LotkaVolterraStats(seed=seed)
g_stats = StubbornGenerator(model=model, prior=prior, summary=summary, seed=seed)

In [ ]:
%%capture
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import timeit
import os

import delfi.distribution as dd
import delfi.inference as infer
import delfi.generator as dg

from delfi.simulator import TwoMoons
import delfi.summarystats as ds
from delfi.utils.viz import plot_pdf, probs2contours

from lfimodels.snl_exps.util import save_results, load_results
from lfimodels.snl_exps.util import draw_sample_uniform_prior_52, load_gt_lv, init_g_lv
from snl.util.plot import plot_hist_marginals

from inspect import getmembers, isclass

fig_path = 'results/figs/'
fontsize = 16

def rasterize_and_save(fname, rasterize_list=None, fig=None, dpi=None,
                       savefig_kw={}):
    """Save a figure with raster and vector components
    This function lets you specify which objects to rasterize at the export
    stage, rather than within each plotting call. Rasterizing certain
    components of a complex figure can significantly reduce file size.
    Inputs
    ------
    fname : str
        Output filename with extension
    rasterize_list : list (or object)
        List of objects to rasterize (or a single object to rasterize)
    fig : matplotlib figure object
        Defaults to current figure
    dpi : int
        Resolution (dots per inch) for rasterizing
    savefig_kw : dict
        Extra keywords to pass to matplotlib.pyplot.savefig
    If rasterize_list is not specified, then all contour, pcolor, and
    collects objects (e.g., ``scatter, fill_between`` etc) will be
    rasterized
    Note: does not work correctly with round=True in Basemap
    Example
    -------
    Rasterize the contour, pcolor, and scatter plots, but not the line
    >>> import matplotlib.pyplot as plt
    >>> from numpy.random import random
    >>> X, Y, Z = random((9, 9)), random((9, 9)), random((9, 9))
    >>> fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows=2)
    >>> cax1 = ax1.contourf(Z)
    >>> cax2 = ax2.scatter(X, Y, s=Z)
    >>> cax3 = ax3.pcolormesh(Z)
    >>> cax4 = ax4.plot(Z[:, 0])
    >>> rasterize_list = [cax1, cax2, cax3]
    >>> rasterize_and_save('out.svg', rasterize_list, fig=fig, dpi=300)
    """

    # Behave like pyplot and act on current figure if no figure is specified
    fig = plt.gcf() if fig is None else fig

    # Need to set_rasterization_zorder in order for rasterizing to work
    zorder = -5  # Somewhat arbitrary, just ensuring less than 0

    if rasterize_list is None:
        # Have a guess at stuff that should be rasterised
        types_to_raster = ['QuadMesh', 'Contour', 'collections']
        rasterize_list = []

        print("""
        No rasterize_list specified, so the following objects will
        be rasterized: """)
        # Get all axes, and then get objects within axes
        for ax in fig.get_axes():
            for item in ax.get_children():
                if any(x in str(item) for x in types_to_raster):
                    rasterize_list.append(item)
        print('\n'.join([str(x) for x in rasterize_list]))
    else:
        # Allow rasterize_list to be input as an object to rasterize
        if type(rasterize_list) != list:
            rasterize_list = [rasterize_list]

    for item in rasterize_list:

        # Whether or not plot is a contour plot is important
        is_contour = (isinstance(item, matplotlib.contour.QuadContourSet) or
                      isinstance(item, matplotlib.tri.TriContourSet))

        # Whether or not collection of lines
        # This is commented as we seldom want to rasterize lines
        # is_lines = isinstance(item, matplotlib.collections.LineCollection)

        # Whether or not current item is list of patches
        all_patch_types = tuple(
            x[1] for x in getmembers(matplotlib.patches, isclass))
        try:
            is_patch_list = isinstance(item[0], all_patch_types)
        except TypeError:
            is_patch_list = False

        # Convert to rasterized mode and then change zorder properties
        if is_contour:
            curr_ax = item.ax.axes
            curr_ax.set_rasterization_zorder(zorder)
            # For contour plots, need to set each part of the contour
            # collection individually
            for contour_level in item.collections:
                contour_level.set_zorder(zorder - 1)
                contour_level.set_rasterized(True)
        elif is_patch_list:
            # For list of patches, need to set zorder for each patch
            for patch in item:
                curr_ax = patch.axes
                curr_ax.set_rasterization_zorder(zorder)
                patch.set_zorder(zorder - 1)
                patch.set_rasterized(True)
        else:
            # For all other objects, we can just do it all at once
            curr_ax = item.axes
            curr_ax.set_rasterization_zorder(zorder)
            item.set_rasterized(True)
            item.set_zorder(zorder - 1)

    # dpi is a savefig keyword argument, but treat it as special since it is
    # important to this function
    if dpi is not None:
        savefig_kw['dpi'] = dpi

    # Save resulting figure
    fig.savefig(fname, **savefig_kw)


In [ ]:
save_path = 'results/lv_validationset/'

plot_seed = 42
exp_id = 'seed' + str(plot_seed)
pars_true, obs_stats = load_gt_lv(init_g_lv(seed=plot_seed))
pnl_a = plot_hist_marginals(samples_rnn, gt=pars_true.flatten(), lims=[-5,2])
pnl_a.set_figwidth(5)
pnl_a.set_figheight(5)
for ax in pnl_a.axes:
    ax.set_xticks([])
    ax.set_yticks([])
#    ax.axis('off')
    
pnl_a.axes[-4].set_xlabel(r'$\theta_1$', fontsize=fontsize)
pnl_a.axes[-3].set_xlabel(r'$\theta_2$', fontsize=fontsize)
pnl_a.axes[-2].set_xlabel(r'$\theta_3$', fontsize=fontsize)    
pnl_a.axes[-1].set_xlabel(r'$\theta_4$', fontsize=fontsize)        
    
PANEL_5A = fig_path +'fig5_a.svg'

savefig_kw = {
'facecolor' : plt.gcf().get_facecolor(), 
'transparent' : True,
'bbox_inches' : 'tight' 
}
rasterize_and_save(PANEL_5A, rasterize_list=[pnl_a.axes[i] for i in [1,3,4,6,7,8]], fig=pnl_a, dpi=600, savefig_kw=savefig_kw)

In [ ]:
save_path = 'results/lv_validationset/'

plot_seed = 42
exp_id = 'seed' + str(plot_seed)
pars_true, obs_stats = load_gt_lv(init_g_lv(seed=plot_seed))
pnl_c = plot_hist_marginals(samples_ss, gt=pars_true.flatten(), lims=[-5,2], upper=True)
pnl_c.set_figwidth(5)
pnl_c.set_figheight(5)
for ax in pnl_c.axes:
    ax.set_xticks([])
    ax.set_yticks([])
#    ax.axis('off')
    
pnl_c.axes[0].set_xlabel(r'$\theta_1$', fontsize=fontsize)
pnl_c.axes[4].set_xlabel(r'$\theta_2$', fontsize=fontsize)
pnl_c.axes[7].set_xlabel(r'$\theta_3$', fontsize=fontsize)    
pnl_c.axes[-1].set_xlabel(r'$\theta_4$', fontsize=fontsize)        
    
PANEL_5C = fig_path +'fig5_c.svg'

savefig_kw = {
'facecolor' : plt.gcf().get_facecolor(), 
'transparent' : True,
'bbox_inches' : 'tight' 
}
rasterize_and_save(PANEL_5C, rasterize_list=[pnl_c.axes[i] for i in [1,2,3,5,6,8]], fig=pnl_c, dpi=600, savefig_kw=savefig_kw)

In [ ]:
import scipy
figh = plt.figure(figsize=(8,5))

round_plot = 10
for k in range(dim_param):
    xx = np.linspace(-5,2,5000) #pars_true[k] + np.linspace(-1., 1., 5000) * 2
    
    ax = plt.subplot(2, 4, k + 1)
    for p in pstats_nonoise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=stats_color, linewidth=1.5)
        plt.xticks([])
        
    #plt.xlabel('$\\theta_{0}$'.format(k))
    if k == 0:
        #plt.ylabel('$q_{F(x,\\phi)}(\\theta)$')
        plt.ylabel('$\\sigma = 0$', fontsize=13)
        plt.ylim([-0.05, 5])
    plt.yticks([])
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom') 

    for p in prnn_nonoise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=rnn_color, linewidth=1.5)
        plt.ylim([-0.05, 5])
        
    ax=plt.subplot(2, 4, 4 + k + 1)
    for p in pstats_noise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=stats_color, linewidth=1.5)
    plt.xlabel('$\\theta_{0}$'.format(k), fontsize=13)
    if k == 0:
        #plt.ylabel('$q_{F(x,\\phi)}(\\theta)$')
        plt.ylabel('$\\sigma = 50$', fontsize=13)
        plt.ylim([-0.05, 5])
    plt.yticks([])
    #plt.box('off')
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')    
    plt.xticks([-5, 2], fontsize=12)


        
    for p in prnn_noise: 
        m = p[round_plot - 1].xs[0].m[k]
        V = p[round_plot - 1].xs[0].S[k,k]
        y = scipy.stats.norm.pdf(xx, m, np.sqrt(V))
        plt.plot(xx, y, color=rnn_color, linewidth=1.5)
        plt.ylim([-0.05, 3])
        
for k in range(dim_param):
    plt.subplot(2, 4, k + 1)
    YL = plt.ylim()
    plt.plot(pars_true[k] * np.ones(2), YL, 'r')
    plt.ylim(YL)
    plt.subplot(2, 4, 4 + k + 1)
    plt.plot(pars_true[k] * np.ones(2), plt.ylim(), 'r')

PANEL_5B = fig_path +'fig5_b.svg'
figh.savefig(PANEL_5B, facecolor=plt.gcf().get_facecolor(), transparent=True, bbox_inches='tight')
figh.show()

In [ ]:
%run -i ./common.ipynb

In [ ]:
# FIGURE and GRID
FIG_HEIGHT_MM = 104
FIG_WIDTH_MM = 80  # set in NIPS2017 notebook to a default value for all figures

FIG_N_ROWS = 2
ROW_1_NCOLS = 2
ROW_1_HEIGHT_MM =      1.2 * (FIG_HEIGHT_MM / FIG_N_ROWS )
ROW_1_WIDTH_COL_1_MM =  1.5* (FIG_WIDTH_MM / ROW_1_NCOLS)

ROW_2_NCOLS = 2
ROW_2_HEIGHT_MM = 0.8 * FIG_HEIGHT_MM / FIG_N_ROWS
ROW_2_WIDTH_COL_1_MM = FIG_WIDTH_MM / ROW_2_NCOLS
ROW_2_WIDTH_COL_2_MM = FIG_WIDTH_MM / ROW_2_NCOLS


fig = create_fig(FIG_WIDTH_MM, FIG_HEIGHT_MM)


fig = add_svg(fig, PANEL_5A, 
              3, 
              0)
fig = add_label(fig, 
                'a)', 
                0, 
                3.5)

fig = add_svg(fig, PANEL_5C, 
              31, 
              0)
fig = add_label(fig, 
                'b)', 
                28, 
                3.5)


fig = add_svg(fig, PANEL_5B, 
              0,
              50) 
fig = add_label(fig, 
                'c)', 
                0, 
                53.5)
fig = add_label(fig, 
                'd)', 
                0, 
                78)



PATH_SVG = PATH_DROPBOX_FIGS + 'fig5.svg'
fig.save(PATH_SVG)
svg(PATH_SVG)
!$INKSCAPE --export-pdf $PATH_DROPBOX_FIGS/fig5.pdf $PATH_SVG
